In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_HOST = os.getenv("QDRANT_HOST")
QDRANT_COLLECTION_NAME = os.getenv("QDRANT_COLLECTION_NAME")

In [2]:
#langsmith - to track my application
LANGCHAIN_TRACING_V2="true"
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

In [3]:
# Document loader
from langchain_community.document_loaders import PyMuPDFLoader, DirectoryLoader

#PyMuPDF fastest in parsling(analyize the text)  #extract images from text in pdf -> ## pip install rapidocr-onnxruntime
loader = DirectoryLoader('information/', glob="*.pdf", loader_cls=PyMuPDFLoader)  
data = loader.load()
# print(data[0])  #- returns with matadata

# Extract text content from the 'Document' objects
texts = [doc.page_content for doc in data]
# print(texts[0])

In [4]:
# # Text Splitter using Semantic chunking - which uses self-attenation concept in Transformer to create independent chunks
from langchain_experimental.text_splitter import SemanticChunker
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Break Points
text_splitter = SemanticChunker(
    embeddings, breakpoint_threshold_type="percentile"
)

docs = text_splitter.create_documents(texts)

In [5]:
print(docs[0])

page_content='Name :  I’m Mahesh.'


In [6]:
# Embidding from HuggingFace

from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cpu'})

In [7]:
# !pip install langchain-chroma
from langchain_chroma import Chroma

db = Chroma.from_documents(docs, embeddings)

In [8]:
# LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    # convert_system_message_to_human=True
    )

In [33]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
Never mention in result - The provided context mentions 
if there is no answer for the question in context jjust replay, Sorry, The chatbot is still on development

<context>
{context}
</context>
Question: {input}""")

In [34]:
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [35]:

from langchain.schema import retriever
from langchain.chains import create_retrieval_chain

retriever=db.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [43]:
response=retrieval_chain.invoke({"input":"resume download link"})
print(response['answer'])

https://drive.google.com/file/d/1CbNVuTRanShwYGzgNtM8zuWRMTX-yYFp/view
